In [1]:
!wget https://www.gutenberg.org/files/1342/1342-0.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!python -m spacy download en_core_web_sm

                                              0.0/12.8 MB ? eta -:--:--
                                              0.3/12.8 MB 8.6 MB/s eta 0:00:02
     -                                        0.5/12.8 MB 10.5 MB/s eta 0:00:02
     --                                       0.9/12.8 MB 9.1 MB/s eta 0:00:02
     ----                                     1.3/12.8 MB 9.5 MB/s eta 0:00:02
     -----                                    1.6/12.8 MB 9.3 MB/s eta 0:00:02
     -----                                    1.9/12.8 MB 9.2 MB/s eta 0:00:02
     ------                                   2.0/12.8 MB 8.1 MB/s eta 0:00:02
     -------                                  2.3/12.8 MB 8.2 MB/s eta 0:00:02
     --------                                 2.7/12.8 MB 8.1 MB/s eta 0:00:02
     ---------                                3.0/12.8 MB 8.0 MB/s eta 0:00:02
     ----------                               3.3/12.8 MB 8.0 MB/s eta 0:00:02
     -----------                              3.6/12.8 MB 


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import os
import spacy

nlp = spacy.load('en_core_web_sm')

def read_file(file_name):
    with open(file_name, encoding="utf8") as file:
        return file.read()
    
text = read_file('1342-1.txt')
processed_text = nlp(text)

In [9]:
# An example of corpus, consists of 7045 sentences
sentences = [s for s in processed_text.sents]

print(len(sentences))

print(len(processed_text.text.split()))

5922
130412


In [10]:
print(sentences[30:34])

[On the other hand,
I, for my part, declare for_ Pride and Prejudice _unhesitatingly., It
seems to me the most perfect, the most characteristic, the most
eminently quintessential of its author’s works; and for this contention
in such narrow space as is permitted to me, I propose here to show
cause._

_, In the first place, the book (it may be barely necessary to remind the
reader) was in its first shape written very early, somewhere about 1796,
when Miss Austen was barely twenty-one; though it was revised and
finished at Chawton some fifteen years later, and was not published till
1813, only four years before her death., I do not know whether, in this
combination of the fresh and vigorous projection of youth, and the
critical revision of middle life, there may be traced the distinct
superiority in point of construction, which, as it seems to me, it
possesses over all the others.]


In [15]:
import gensim
from gensim.models import Word2Vec

print(f"Gensim Version: {gensim.__version__}")

# We need data for training the model
processed_sentences = [sent.lemma_.split() for sent in processed_text.sents]

processed_sentences[0]

Gensim Version: 3.8.0


['\ufeffthe',
 'Project',
 'Gutenberg',
 'eBook',
 'of',
 'Pride',
 'and',
 'prejudice,',
 'by',
 'Jane',
 'Austen',
 'this',
 'eBook',
 'be',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'in',
 'the',
 'United',
 'States',
 'and',
 'most',
 'other',
 'part',
 'of',
 'the',
 'world',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restriction',
 'whatsoever.']

In [16]:
# Word2Vec accepts several parameters that affect both training speed and quality
interchangeable_words_model = Word2Vec(
    sentences=processed_sentences,
    min_count=10, # Purning the internal dictionary
    size=200, # the number of dimensions (N) gensim maps the word onto
    window=2, # Define when two words are together, 2 means, 2 words left and 2 words right
    compute_loss=True,
    sg=1
)
print(len(interchangeable_words_model.wv.vocab))

# getting the training loss
training_loss = interchangeable_words_model.get_latest_training_loss()
print(f"Training Loss: {training_loss}")

for w, sim in interchangeable_words_model.wv.most_similar('Darcy'):
    print((w, sim))

d:\Softwares\Python311\Lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


1290
Training Loss: 2522556.58985487
('Collins', 0.9899777173995972)
('Bingley', 0.9888432025909424)
('Gardiner', 0.9602433443069458)
('Darcy,', 0.9585365653038025)
('Bennet', 0.9566755294799805)
('Bennet,', 0.9492521286010742)
('Wickham', 0.9396989345550537)
('Collins,', 0.9186433553695679)
('Bingley,', 0.9144344329833984)
('Darcy’s', 0.8991830348968506)


In [17]:
from gensim.models import FastText

model = FastText(window=2)
model.build_vocab(sentences=processed_sentences)
model.train(sentences=processed_sentences, total_examples=len(processed_sentences), epochs=10)

for w, sim in model.wv.most_similar('Darcy'):
    print((w, sim))

('Darcy!', 0.9988260269165039)
('Darcy;', 0.997593879699707)
('Darcy,"', 0.997324526309967)
('Darcy,', 0.9957572817802429)
('Darcy?"', 0.9950392246246338)
('Darcy.', 0.9946999549865723)
('Darcy."', 0.9944714307785034)
('Darcy’s', 0.9861998558044434)
('Collins', 0.9821525812149048)
('Wickham', 0.9810268878936768)


In [ ]:
for w, sim in model.wv.most_similar('Darcy', topn=20):
    print((w, sim))

In [ ]:
model = FastText(window=50)
model.build_vocab(sentences=processed_sentences)
model.train(
    sentences=processed_sentences, 
    total_examples=len(processed_sentences), 
    epochs=10
)

for w, sim in model.wv.most_similar('Darcy'):
    print((w, sim))

In [19]:
print("night" in model.wv.vocab)
print("nights" in model.wv.vocab)

print(model.wv.similarity("nights", "night"))
print(model.wv.similarity("tonight", "night"))

True
False
0.9966166
0.9979609


In [38]:
print(model.wv.similarity("tiger", "peacock"))

0.982736
